In [16]:
import pandas as pd

In [17]:
googleSheetId = '1BGNomIEkIgiPaPbMpeCArVCpERKK9TGHYFyT4Mt9grk'
worksheetName = 'sheet1'
URL = f'https://docs.google.com/spreadsheets/d/{googleSheetId}/gviz/tq?tqx=out:csv&sheet={worksheetName}'
inscricoes = pd.read_csv(URL)

In [18]:
googleSheetId = '1bNqrMwDAPVNG8YxzEJt6sZNmtO9xEdHjIvDUw8c_oyY'
worksheetName = 'sheet1'
URL = f'https://docs.google.com/spreadsheets/d/{googleSheetId}/gviz/tq?tqx=out:csv&sheet={worksheetName}'
cancelamentos = pd.read_csv(URL)

In [19]:
inscricoes.Timestamp = pd.to_datetime(inscricoes.Timestamp)
cancelamentos.Timestamp = pd.to_datetime(cancelamentos.Timestamp)

In [20]:
inscricoes.columns

Index(['Timestamp', 'E-mail', 'Consequência do acidente',
       'Abrangência geográfica', 'UF', 'UORG', 'Tipo de acidente',
       'Setores econômicos', 'Seção CNAE', 'Fator de risco',
       'Fatores de risco'],
      dtype='object')

In [21]:
ultimo_cancelamento = cancelamentos.sort_values('Timestamp', ascending=False).groupby('E-mail').head(1)

In [23]:
compilado = inscricoes.merge(ultimo_cancelamento, 
                             how='left', 
                             on='E-mail',
                             suffixes=['_incricao', '_cancelamento'])

In [24]:
compilado

,Timestamp_incricao,E-mail,Consequência do acidente,Abrangência geográfica,UF,UORG,Tipo de acidente,Setores econômicos,Seção CNAE,Fator de risco,Fatores de risco,Timestamp_cancelamento
0,2022-07-27 16:55:25,joao.reis@economia.gov.br,Óbito,Unidade Organizacional (ex.: Gerência de Vargi...,NaN,AM - 025901000 - GRTE/TABATINGA/AM,Acidentes típicos,Não,NaN,Não,NaN,2022-07-22 09:23:43
1,2022-07-27 16:56:51,joao.reis@economia.gov.br,"Amputação de membro (exceto dedo), Amputação d...",Unidade da Federação (ex.: MG),CE,NaN,"Acidentes típicos, Doenças do Trabalho",Sim,"B - Indústrias Extrativas, C - Indústrias de T...",Sim,611 - Máquinas e equipamentos,2022-07-22 09:23:43
2,2022-07-27 17:45:41,joao.reis@economia.gov.br,Óbito,Unidade da Federação (ex.: MG),AC,NaN,"Acidentes típicos, Doenças do Trabalho",Não,NaN,Não,NaN,2022-07-22 09:23:43
3,2022-07-27 17:47:33,joao.reis@economia.gov.br,Perda de audição,Unidade Organizacional (ex.: Gerência de Vargi...,NaN,AM - 025901000 - GRTE/TABATINGA/AM,Acidentes típicos,Sim,D - Eletricidade e Gás,Sim,132 - Radiação ionizante,2022-07-22 09:23:43


In [25]:
compilado['intervalo'] = compilado.Timestamp_cancelamento - compilado.Timestamp_incricao

In [26]:
compilado

,Timestamp_incricao,E-mail,Consequência do acidente,Abrangência geográfica,UF,UORG,Tipo de acidente,Setores econômicos,Seção CNAE,Fator de risco,Fatores de risco,Timestamp_cancelamento,intervalo
0,2022-07-27 16:55:25,joao.reis@economia.gov.br,Óbito,Unidade Organizacional (ex.: Gerência de Vargi...,NaN,AM - 025901000 - GRTE/TABATINGA/AM,Acidentes típicos,Não,NaN,Não,NaN,2022-07-22 09:23:43,-6 days +16:28:18
1,2022-07-27 16:56:51,joao.reis@economia.gov.br,"Amputação de membro (exceto dedo), Amputação d...",Unidade da Federação (ex.: MG),CE,NaN,"Acidentes típicos, Doenças do Trabalho",Sim,"B - Indústrias Extrativas, C - Indústrias de T...",Sim,611 - Máquinas e equipamentos,2022-07-22 09:23:43,-6 days +16:26:52
2,2022-07-27 17:45:41,joao.reis@economia.gov.br,Óbito,Unidade da Federação (ex.: MG),AC,NaN,"Acidentes típicos, Doenças do Trabalho",Não,NaN,Não,NaN,2022-07-22 09:23:43,-6 days +15:38:02
3,2022-07-27 17:47:33,joao.reis@economia.gov.br,Perda de audição,Unidade Organizacional (ex.: Gerência de Vargi...,NaN,AM - 025901000 - GRTE/TABATINGA/AM,Acidentes típicos,Sim,D - Eletricidade e Gás,Sim,132 - Radiação ionizante,2022-07-22 09:23:43,-6 days +15:36:10


In [27]:
from datetime import timedelta
compilado['vigente'] = compilado.intervalo < timedelta(microseconds=0)

In [28]:
compilado[compilado.vigente == True]

,Timestamp_incricao,E-mail,Consequência do acidente,Abrangência geográfica,UF,UORG,Tipo de acidente,Setores econômicos,Seção CNAE,Fator de risco,Fatores de risco,Timestamp_cancelamento,intervalo,vigente
0,2022-07-27 16:55:25,joao.reis@economia.gov.br,Óbito,Unidade Organizacional (ex.: Gerência de Vargi...,NaN,AM - 025901000 - GRTE/TABATINGA/AM,Acidentes típicos,Não,NaN,Não,NaN,2022-07-22 09:23:43,-6 days +16:28:18,True
1,2022-07-27 16:56:51,joao.reis@economia.gov.br,"Amputação de membro (exceto dedo), Amputação d...",Unidade da Federação (ex.: MG),CE,NaN,"Acidentes típicos, Doenças do Trabalho",Sim,"B - Indústrias Extrativas, C - Indústrias de T...",Sim,611 - Máquinas e equipamentos,2022-07-22 09:23:43,-6 days +16:26:52,True
2,2022-07-27 17:45:41,joao.reis@economia.gov.br,Óbito,Unidade da Federação (ex.: MG),AC,NaN,"Acidentes típicos, Doenças do Trabalho",Não,NaN,Não,NaN,2022-07-22 09:23:43,-6 days +15:38:02,True
3,2022-07-27 17:47:33,joao.reis@economia.gov.br,Perda de audição,Unidade Organizacional (ex.: Gerência de Vargi...,NaN,AM - 025901000 - GRTE/TABATINGA/AM,Acidentes típicos,Sim,D - Eletricidade e Gás,Sim,132 - Radiação ionizante,2022-07-22 09:23:43,-6 days +15:36:10,True
